## Notebook for spike sorting from .kwd data using the Pipeline of mountainsort (linux channel has pipeline)
Uses:
    - intan2kwik (https://github.com/zekearneodo/intan2kwik/blob/master/README.md)
    - mountainlab suite(https://github.com/flatironinstitute/mountainlab-js)
    - mountainsort https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md
    - mountainsort examples https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md

In [1]:
import socket
import os
import glob
import json
from typing import Union
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import subprocess
from datetime import timedelta
from importlib import reload

# pipeline imports
from pipefinch.neural.convert import intan
from pipefinch.neural.sort.mountain import core as msc
from pipefinch.h5tools.kwik import kutil
from pipefinch.pipeline import probes


from pipefinch.h5tools.kwik import kwdfunctions as kwdf

from intan2kwik import kwd

#mountainsort imports (for sorting)
#import mountainlab_pytools.mlproc as mlp

import logging

# Setup the logger
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
ch.setFormatter(formatter)
logger.addHandler(ch)
        
logger.info('Logger set')
logger.info('Hostname {}'.format(socket.gethostname()))

2019-04-23 15:39:38,179 - root - INFO - Logger set
2019-04-23 15:39:38,179 - root - INFO - Hostname lookfar


### Session parameters and raw files

In [2]:
from pipefinch.neural.sort import kilo

In [4]:
from pipefinch.pipeline import filestructure as et
reload(et)
reload(kwd)

# sess_par = {'bird': 'p14r14',
#            'sess': '2019-02-14_2250_01'}
sess_par = {'bird': 'p14r14',
           'sess': '2019-02-15_3125_01',
           'probe': 'probe_0' # probe to sort ('probe_0', 'probe_1') (to lookup in the rig_par which port to extract)
           }

exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'])

sort_params = {'adjacency_radius': -1,
              'detect_threshold': 2,
              'freq_min': 600}

ds_params = {'detect_sign': -1}

ks_params = {'use_gpu': 1,
            'auto_merge': 1,
            'filt_per_chan': 4,
            }

# convenient paths
kwik_folder = exp_struct['folders']['kwik']
msort_folder = exp_struct['folders']['msort']
raw_folder = exp_struct['folders']['raw']
kwd_path = exp_struct['files']['kwd']
bin_path = exp_struct['files']['mda_raw']

In [5]:
exp_struct

{'folders': {'raw': '/Volumes/Samsung_X5/microdrive/p14r14/Ephys/raw/2019-02-15_3125_01',
  'kwik': '/Volumes/Samsung_X5/microdrive/p14r14/Ephys/kwik/2019-02-15_3125_01',
  'msort': '/Volumes/Samsung_X5/scratch/p14r14/Ephys/msort/2019-02-15_3125_01',
  'ksort': '/Volumes/Samsung_X5/scratch/p14r14/Ephys/ksort/2019-02-15_3125_01'},
 'files': {'par': '/Volumes/Samsung_X5/scratch/p14r14/Ephys/ksort/2019-02-15_3125_01/params.json',
  'set': '/Volumes/Samsung_X5/microdrive/p14r14/Ephys/raw/2019-02-15_3125_01/settings.isf',
  'rig': '/Volumes/Samsung_X5/microdrive/p14r14/Ephys/raw/2019-02-15_3125_01/rig.json',
  'kwd': '/Volumes/Samsung_X5/microdrive/p14r14/Ephys/kwik/2019-02-15_3125_01/streams.kwd',
  'kwik': '/Volumes/Samsung_X5/microdrive/p14r14/Ephys/kwik/2019-02-15_3125_01/spikes.kwik',
  'kwe': '/Volumes/Samsung_X5/microdrive/p14r14/Ephys/kwik/2019-02-15_3125_01/events.kwe',
  'mda_raw': '/Volumes/Samsung_X5/scratch/p14r14/Ephys/msort/2019-02-15_3125_01/raw.mda',
  'bin_raw': '/Volumes/

### convert the whole session to a .kwd file
Conversion sends every .rhd file in the folder to a rec in the .kwd file (experiment.kwd in the session ss folder)
All of the files and all of the channels are converted; filtering and subselection of sub-epochs and channels occurs later.
The .kwd is raw data, only in a friendlier format.


#### Make a file for the session for the first time

In [6]:
reload(kwd)
## Convert the whole session to a kwd file
os.makedirs(kwik_folder, exist_ok=True)
first_intan_hdr, sess_pd = kwd.intan_to_kwd(raw_folder, kwd_path)

2019-04-23 15:39:55,485 - intan2kwik.kwd - INFO - reading intan chans data across all of rec /Volumes/Samsung_X5/microdrive/p14r14/Ephys/raw/2019-02-15_3125_01
2019-04-23 15:39:55,504 - intan2kwik.kwd - INFO - Found 5 .rhd files split in 5 recordings
2019-04-23 15:39:55,505 - intan2kwik.kwd - INFO - dest file: /Volumes/Samsung_X5/microdrive/p14r14/Ephys/kwik/2019-02-15_3125_01/streams.kwd


2019-04-23 15:40:55,875 - intan2kwik.kwd - INFO - moving back to /Volumes/Samsung_X5/microdrive/p14r14/Ephys/kwik/2019-02-15_3125_01/streams.kwd


2019-04-23 15:40:56,890 - intan2kwik.kwd - INFO - removing temp file


# play with glxtokwik

In [155]:
from intan2kwik.core.file import util as fu
import datetime
import time
reload(fu)

<module 'intan2kwik.core.file.util' from '/Users/zeke/repos/intan2kwik/intan2kwik/core/file/util.py'>

In [112]:
sess_pd

,path,f_name,t_stamp,t_diff,rec_break,rec
0,/Volumes/Samsung_X5/microdrive/p14r14/Ephys/ra...,raw_190215_102323.rhd,2019-02-15 10:23:23,NaT,False,0
1,/Volumes/Samsung_X5/microdrive/p14r14/Ephys/ra...,raw_190215_102738.rhd,2019-02-15 10:27:38,00:04:15,True,1
2,/Volumes/Samsung_X5/microdrive/p14r14/Ephys/ra...,raw_190215_103028.rhd,2019-02-15 10:30:28,00:02:50,True,2
3,/Volumes/Samsung_X5/microdrive/p14r14/Ephys/ra...,raw_190215_103455.rhd,2019-02-15 10:34:55,00:04:27,True,3
4,/Volumes/Samsung_X5/microdrive/p14r14/Ephys/ra...,raw_190215_103745.rhd,2019-02-15 10:37:45,00:02:50,True,4


In [156]:
def read_chan_order(meta_parse) -> pd.DataFrame:
    # meta_parse is the ini parameter file, read with read_headless_config
    # we want a three col array with [ch_name, order_in_group, order_in_total]
    chans = meta_parse.get('root', '~snsChanMap')
    chans_tuple_list = lf_chans.split(')(')[1:]
    chans_tuple_list[-1]=chans_tuple_list[-1].split(')')[0]
    all_tuples = [parse.parse('{};{}:{}', x).fixed for x in chans_tuple_list]
    return pd.DataFrame(atp, columns=['name', 'order', 'abs_order'])


atp = read_chan_order(cp)


In [157]:
atp.head(5)

,name,order,abs_order
0,LF0,384,384
1,LF1,385,385
2,LF2,386,386
3,LF3,387,387
4,LF4,388,388


In [160]:
atp['name'].values

array(['LF0', 'LF1', 'LF2', 'LF3', 'LF4', 'LF5', 'LF6', 'LF7', 'LF8',
       'LF9', 'LF10', 'LF11', 'LF12', 'LF13', 'LF14', 'LF15', 'LF16',
       'LF17', 'LF18', 'LF19', 'LF20', 'LF21', 'LF22', 'LF23', 'LF24',
       'LF25', 'LF26', 'LF27', 'LF28', 'LF29', 'LF30', 'LF31', 'LF32',
       'LF33', 'LF34', 'LF35', 'LF36', 'LF37', 'LF38', 'LF39', 'LF40',
       'LF41', 'LF42', 'LF43', 'LF44', 'LF45', 'LF46', 'LF47', 'LF48',
       'LF49', 'LF50', 'LF51', 'LF52', 'LF53', 'LF54', 'LF55', 'LF56',
       'LF57', 'LF58', 'LF59', 'LF60', 'LF61', 'LF62', 'LF63', 'LF64',
       'LF65', 'LF66', 'LF67', 'LF68', 'LF69', 'LF70', 'LF71', 'LF72',
       'LF73', 'LF74', 'LF75', 'LF76', 'LF77', 'LF78', 'LF79', 'LF80',
       'LF81', 'LF82', 'LF83', 'LF84', 'LF85', 'LF86', 'LF87', 'LF88',
       'LF89', 'LF90', 'LF91', 'LF92', 'LF93', 'LF94', 'LF95', 'LF96',
       'LF97', 'LF98', 'LF99', 'LF100', 'LF101', 'LF102', 'LF103',
       'LF104', 'LF105', 'LF106', 'LF107', 'LF108', 'LF109', 'LF110',
       'LF111

In [154]:
pd.DataFrame(atp, columns=['name', 'order', 'abs_order'])

,name,order,abs_order
0,LF0,384,384
1,LF1,385,385
2,LF2,386,386
3,LF3,387,387
4,LF4,388,388
5,LF5,389,389
6,LF6,390,390
7,LF7,391,391
8,LF8,392,392
9,LF9,393,393


In [135]:
chans_tuple_list = lf_chans.split(')(')

In [138]:
chans_tuple_list[-1]=chans_tuple_list[-1].split(')')[0]

In [148]:
rs = parse.parse('{};{}:{}', chans_tuple_list[-1]).fixed
rs

('SY0', '768', '768')

In [119]:
lf_chans = cp.get('root', '~snsChanMap')

In [109]:
datetime.datetime.strptime(time_str, "%Y-%m-%dT%H:%M:%S")

datetime.datetime(2016, 7, 22, 18, 15, 30)

#### Update a session with subsequently recorded rhd files

In [57]:
reload(kwdf)
_, nu_pd, _ = kwdf.update_kwd(kwd_path, raw_folder)

2019-04-27 00:57:00,740 - pipefinch.h5tools.kwik.kwdfunctions - INFO - updating kwd file /Volumes/Samsung_X5/microdrive/p14r14/Ephys/kwik/2019-02-15_3125_01/streams.kwd from folder /Volumes/Samsung_X5/microdrive/p14r14/Ephys/raw/2019-02-15_3125_01
2019-04-27 00:57:00,882 - pipefinch.h5tools.kwik.kwdfunctions - INFO - No new files to add to the file


## Make .mda file with a set of recordings in a session
 - pick all in port A
 - get all rec within a time range


In [8]:
# get the session meta
pd_meta = kwdf.get_all_rec_meta(kwd_path)
pd_meta.head()


,bit_depth,name,sample_rate,start_sample,start_time,channel_bit_volts,channel_names,channels_sample_rate,dig_channel_names,is_multiSampleRate_data,valid_samples,samples_count
0,16,0,20000.0,0,2019-02-15 10:23:23,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[A-000, A-001, A-002, A-003, A-004, A-005, A-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...","[DIN-00, DIN-01]",0,"[3484080.0, 3484080.0, 3484080.0, 3484080.0, 3...",3484080
1,16,1,20000.0,0,2019-02-15 10:27:38,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[A-000, A-001, A-002, A-003, A-004, A-005, A-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...","[DIN-00, DIN-01]",0,"[1997280.0, 1997280.0, 1997280.0, 1997280.0, 1...",1997280
2,16,2,20000.0,0,2019-02-15 10:30:28,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[A-000, A-001, A-002, A-003, A-004, A-005, A-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...","[DIN-00, DIN-01]",0,"[4657680.0, 4657680.0, 4657680.0, 4657680.0, 4...",4657680
3,16,3,20000.0,0,2019-02-15 10:34:55,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[A-000, A-001, A-002, A-003, A-004, A-005, A-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...","[DIN-00, DIN-01]",0,"[3566160.0, 3566160.0, 3566160.0, 3566160.0, 3...",3566160
4,16,4,20000.0,0,2019-02-15 10:37:45,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[A-000, A-001, A-002, A-003, A-004, A-005, A-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...","[DIN-00, DIN-01]",0,"[5299200.0, 5299200.0, 5299200.0, 5299200.0, 5...",5299200


### pick a time interval of the recordings

In [9]:
def select_time_span(meta_pd, start, span_minutes):
    end = start + timedelta(minutes=span_minutes)
    pd_selection = meta_pd.loc[meta_pd['start_time'].between(start, end)]
    return pd_selection

pd_meta_selection = select_time_span(pd_meta, pd_meta['start_time'][0], 240)

In [10]:
# for instance
pd_meta_selection.head()

,bit_depth,name,sample_rate,start_sample,start_time,channel_bit_volts,channel_names,channels_sample_rate,dig_channel_names,is_multiSampleRate_data,valid_samples,samples_count
0,16,0,20000.0,0,2019-02-15 10:23:23,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[A-000, A-001, A-002, A-003, A-004, A-005, A-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...","[DIN-00, DIN-01]",0,"[3484080.0, 3484080.0, 3484080.0, 3484080.0, 3...",3484080
1,16,1,20000.0,0,2019-02-15 10:27:38,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[A-000, A-001, A-002, A-003, A-004, A-005, A-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...","[DIN-00, DIN-01]",0,"[1997280.0, 1997280.0, 1997280.0, 1997280.0, 1...",1997280
2,16,2,20000.0,0,2019-02-15 10:30:28,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[A-000, A-001, A-002, A-003, A-004, A-005, A-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...","[DIN-00, DIN-01]",0,"[4657680.0, 4657680.0, 4657680.0, 4657680.0, 4...",4657680
3,16,3,20000.0,0,2019-02-15 10:34:55,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[A-000, A-001, A-002, A-003, A-004, A-005, A-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...","[DIN-00, DIN-01]",0,"[3566160.0, 3566160.0, 3566160.0, 3566160.0, 3...",3566160
4,16,4,20000.0,0,2019-02-15 10:37:45,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[A-000, A-001, A-002, A-003, A-004, A-005, A-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...","[DIN-00, DIN-01]",0,"[5299200.0, 5299200.0, 5299200.0, 5299200.0, 5...",5299200


In [11]:
# load the rig parameters
rig_par_file = exp_struct['files']['rig']
with open(rig_par_file, 'r') as fp:
    rig_par = json.load(fp)

# get the probe and the port where the probe was connected
selected_probe = sess_par['probe']
probe_port = rig_par['chan']['port'][selected_probe].strip('-')

# get the channel indices of the probe's port
wanted_chans = np.array([probe_port + '-'])  # all ephys channels

chan_list = kwdf.get_all_chan_names(pd_meta_selection, chan_filt=wanted_chans)

#all_rec_list = kutil.get_rec_list(exp_struct['files']['kwd'])
selection_rec_list = np.unique(pd_meta_selection['name'])

rec_chans = pd_meta.loc[pd_meta['name'] == selection_rec_list[0], 'channel_names'].values
rec_chans_idx = kwdf.find_chan_names_idx(rec_chans[0], chan_list)

# make the mda binary file
bin_path = exp_struct['files']['bin_raw']
os.makedirs(exp_struct['folders']['ksort'], exist_ok=True)
bin_file = kwdf.kwd_to_binary(exp_struct['files']['kwd'],
                              exp_struct['files']['bin_raw'],
                              chan_list=chan_list,
                              rec_list=selection_rec_list, header='bin')


2019-04-23 15:41:14,472 - pipefinch.h5tools.kwik.kwdfunctions - INFO - Writing kwd_file /Volumes/Samsung_X5/microdrive/p14r14/Ephys/kwik/2019-02-15_3125_01/streams.kwd to binary
2019-04-23 15:41:14,492 - pipefinch.h5tools.kwik.kwdfunctions - INFO - Channels to extract: ['A-000' 'A-001' 'A-002' 'A-003' 'A-004' 'A-005' 'A-006' 'A-007' 'A-008'
 'A-009' 'A-010' 'A-011' 'A-012' 'A-013' 'A-014' 'A-015' 'A-016' 'A-017'
 'A-018' 'A-019' 'A-020' 'A-021' 'A-022' 'A-023' 'A-024' 'A-025' 'A-026'
 'A-027' 'A-028' 'A-029' 'A-030' 'A-031']
2019-04-23 15:41:14,493 - pipefinch.h5tools.kwik.kwdfunctions - INFO - Will go through recs [0 1 2 3 4]
2019-04-23 15:41:14,494 - pipefinch.h5tools.kwik.kwdfunctions - INFO - Creating binary file /Volumes/Samsung_X5/scratch/p14r14/Ephys/ksort/2019-02-15_3125_01/raw.bin


2019-04-23 15:41:25,382 - pipefinch.h5tools.kwik.kwdfunctions - INFO - 608140800 elements written


## Scripts for sorting with Kilosort
Steps involved:
 - Make binary file with selected recs, chans
 - Set kilosort parameters
 - Make kilosort chanmap
 - Make kilosort scripts and phy parameters file (for manual curation)
 - Run the kilosort scripts (via matlab)
 - Expose the paths for manual curation
 - After curation, make the kwik file with sorted data
 - Cleanup and move metadata to permanentt locations

### prep the files with their nice formats, locations and names


In [12]:
from pipefinch.neural.sort.kilo import core as ksc

In [13]:
reload(ksc)
ks_params = {'use_gpu': 1,
            'auto_merge': 1,
            'filt_per_chan': 4,
            's_f': 30000,
            'n_chan': 32}

In [27]:
reload(kwdf)

<module 'pipefinch.h5tools.kwik.kwdfunctions' from '/Users/zeke/repos/pipefinch/pipefinch/h5tools/kwik/kwdfunctions.py'>

In [28]:
kwdf.get_data_type(exp_struct['files']['kwd'])

dtype('int16')

In [14]:
exp_struct['folders']['ksort']

'/Volumes/Samsung_X5/scratch/p14r14/Ephys/ksort/2019-02-15_3125_01'

In [49]:
reload(ksc)
reload(probes)
file_paths, out_folder = ksc.make_paths(exp_struct['folders']['ksort'])
#os.makedirs(exp_struct['folders']['msort'], exist_ok=True)

# make the probe file
rec_chans = pd_meta.loc[pd_meta['name']==0, 'channel_names'].values
rec_chans_idx = kwdf.find_chan_names_idx(rec_chans[0], chan_list)
probe = rig_par['probe'][selected_probe]['model']
headstage = rig_par['probe'][selected_probe]['headstage']
probe_chans = rec_chans_idx - np.min(rec_chans_idx)

probe_dict = probes.make_map(probe, probe_chans, return_dict=True)
kilo_chan_map = ksc.make_kilo_chanmap(probe_dict, file_paths['prb'])

# parameters to pass to the msort scripts, other than de defaults
ks_params.update({'s_f': int(kwdf.get_sampling_rate(pd_meta, 0)), # required,
                  'n_chan': probe_chans.size, # total number of chans in the .bin file,
                  'dtype_name': kwdf.get_data_type(exp_struct['files']['kwd']).name
            })
ksc.make_kilo_scripts(exp_struct['folders']['ksort'], ks_params)
phy_pars = ksc.make_phy_par_file(ks_params, file_paths)

2019-04-23 17:23:31,702 - pipefinch.neural.sort.kilo.core - INFO - Written kilo script /Volumes/Samsung_X5/scratch/p14r14/Ephys/ksort/2019-02-15_3125_01/master.m
2019-04-23 17:23:31,706 - pipefinch.neural.sort.kilo.core - INFO - Written kilo script /Volumes/Samsung_X5/scratch/p14r14/Ephys/ksort/2019-02-15_3125_01/config.m
2019-04-23 17:23:31,709 - pipefinch.neural.sort.kilo.core - INFO - Written phy parameters file /Volumes/Samsung_X5/scratch/p14r14/Ephys/ksort/2019-02-15_3125_01/params.py


In [50]:
file_paths

{'bin': '/Volumes/Samsung_X5/scratch/p14r14/Ephys/ksort/2019-02-15_3125_01/raw.bin',
 'params': '/Volumes/Samsung_X5/scratch/p14r14/Ephys/ksort/2019-02-15_3125_01/params.json',
 'prb': '/Volumes/Samsung_X5/scratch/p14r14/Ephys/ksort/2019-02-15_3125_01/chanMap.mat',
 'rez': '/Volumes/Samsung_X5/scratch/p14r14/Ephys/ksort/2019-02-15_3125_01/rez2.mat',
 'mat_log': '/Volumes/Samsung_X5/scratch/p14r14/Ephys/ksort/2019-02-15_3125_01/kilosort_mat.log',
 'phy_par': '/Volumes/Samsung_X5/scratch/p14r14/Ephys/ksort/2019-02-15_3125_01/params.py'}

In [52]:
reload(ksc)
ksc.do_the_sort(file_paths)

2019-04-23 17:34:50,661 - pipefinch.neural.sort.kilo.core - INFO - Running kilosort on matlab
2019-04-23 17:34:50,661 - pipefinch.neural.sort.kilo.core - INFO - Sort folder is /Volumes/Samsung_X5/scratch/p14r14/Ephys/ksort/2019-02-15_3125_01
2019-04-23 17:34:50,662 - pipefinch.neural.sort.kilo.core - INFO - output to /Volumes/Samsung_X5/scratch/p14r14/Ephys/ksort/2019-02-15_3125_01/kilosort_mat.log


FileNotFoundError: [Errno 2] No such file or directory: 'matlab': 'matlab'

In [16]:
output_dir = os.path.join(exp_struct['folders']['msort'], 'sort_out');
# Pipeline was failing, need to debug this ('NameError: name 'widgets' is not defined')
# with Pipeline:
#     msc.sort_dataset(file_paths=file_paths, adjacency_radius=-1, detect_threshold=3, dispatch_method='add')

msc.sort_dataset(file_paths=file_paths, **sort_params);

2019-02-22 14:59:18,248 - pipefinch.sort.mountain.comre - INFO - Bandpass filter


RUNNING: ml-run-process ephys.bandpass_filter --inputs timeseries:/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-15_3125_0102/raw.mda --parameters freq_max:6000 freq_min:600 samplerate:20000 --outputs timeseries_out:/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-15_3125_0102/sort_out/filt.mda.prv


2019-02-22 15:03:39,760 - pipefinch.sort.mountain.comre - INFO - Whitening


[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
Process signature: 6fa90314bd60526260e249eaf7c88fb5a53b740e
[ Checking outputs... ]
{"timeseries_out":"/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-15_3125_0102/sort_out/filt.mda.prv"}
Processing ouput - /media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-15_3125_0102/sort_out/filt.mda.prv
false
{"timeseries_out":"/media/zinch/Windows/experiment/msort-tmp/output_6fa90314bd60526260e249eaf7c88fb5a53b740e_timeseries_out.mda"}
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Creating links to input files... ]
[ Preparing temporary outputs... ]
Processing ouput - /media/zinch/Windows/experiment/msort-tmp/output_6fa90314bd60526260e249eaf7c88fb5a53b740e_timeseries_out.mda
false
[ Initializing process ... ]
[ Running ... ] /home/zinch/conda/envs/msort/bin/python3 /home/zinch/conda/envs/msort/etc/mountainlab/packages/ml_ephys/preprocessing/pre

2019-02-22 15:07:38,274 - pipefinch.sort.mountain.comre - INFO - Sorting


[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
Process signature: 40459222a70a1577e2eaf27302d72d93dd553dab
[ Checking outputs... ]
{"timeseries_out":"/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-15_3125_0102/sort_out/pre.mda.prv"}
Processing ouput - /media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-15_3125_0102/sort_out/pre.mda.prv
false
{"timeseries_out":"/media/zinch/Windows/experiment/msort-tmp/output_40459222a70a1577e2eaf27302d72d93dd553dab_timeseries_out.mda"}
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Creating links to input files... ]
[ Preparing temporary outputs... ]
Processing ouput - /media/zinch/Windows/experiment/msort-tmp/output_40459222a70a1577e2eaf27302d72d93dd553dab_timeseries_out.mda
false
[ Initializing process ... ]
[ Running ... ] /home/zinch/conda/envs/msort/bin/python3 /home/zinch/conda/envs/msort/etc/mountainlab/packages/ml_ephys/preprocessing/prepr

2019-02-22 16:12:15,210 - pipefinch.sort.mountain.comre - INFO - Getting cluster metrics


[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
Process signature: f6959c75fc9c091d895f7fef9038a9bcea0094bf
[ Checking outputs... ]
{"firings_out":"/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-15_3125_0102/sort_out/firings.mda"}
Processing ouput - /media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-15_3125_0102/sort_out/firings.mda
false
{"firings_out":"/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-15_3125_0102/sort_out/firings.mda"}
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Creating links to input files... ]
[ Preparing temporary outputs... ]
Processing ouput - /media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-15_3125_0102/sort_out/firings.mda
false
[ Initializing process ... ]
[ Running ... ] /home/zinch/conda/envs/msort/bin/python3 /home/zinch/conda/envs/msort/etc/mountainlab/packages/ml_ms4alg/ms4alg_spec.py.mp ms4alg.sort --_tempdir=/media/zinch/Wi

2019-02-22 16:24:33,679 - pipefinch.sort.mountain.comre - INFO - Automatically curating


[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
Process signature: a90d63ef1d16bb5d7c873cc75bc766b7e35dee0d
[ Checking outputs... ]
{"metrics_out":"/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-15_3125_0102/sort_out/cluster_metrics.json"}
Processing ouput - /media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-15_3125_0102/sort_out/cluster_metrics.json
false
{"metrics_out":"/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-15_3125_0102/sort_out/cluster_metrics.json"}
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Creating links to input files... ]
[ Preparing temporary outputs... ]
Processing ouput - /media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-15_3125_0102/sort_out/cluster_metrics.json
false
[ Initializing process ... ]
[ Running ... ] /home/zinch/conda/envs/msort/etc/mountainlab/packages/ms3.mp ms3.combine_cluster_metrics --_tempdir=/media/zinch/Windows/expe

In [17]:
file_paths

{'mda': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-15_3125_0102/raw.mda',
 'params': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-15_3125_0102/params.json',
 'geom': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-15_3125_0102/geom.csv',
 'filt': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-15_3125_0102/sort_out/filt.mda.prv',
 'pre': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-15_3125_0102/sort_out/pre.mda.prv',
 'firings': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-15_3125_0102/sort_out/firings.mda',
 'firings_curated': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-15_3125_0102/sort_out/firings_curated.mda',
 'cluster_metrics': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-15_3125_0102/sort_out/cluster_metrics.json',
 'cluster_metrics_curated': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-15_3125_0102/sort_out/cluster_metrics_curated.json'}

## Command for viewing:
 - open up terminal with the environment msort
 - go go the ss_data folder for the session
 - run the command: qt-mountainview --raw raw.mda --filt sort_out/filt.mda.prv --pre sort_out/pre.mda.prv --samplerate=20000 --firings sort_out/firings.mda --cluster_metrics sort_out/cluster_metrics.json

# After manual curation
 - save the curated spikes in the sort_out/firings_curated.mda
 - come back to the notebook and run 

In [18]:
from pipefinch.h5tools.kwik import kwikfunctions as kwkf
reload(kwkf)
reload(et)
firings_to_save = 'firings_curated' # 'curated' or 'firings' for default_output


metrics_to_save = 'cluster_metrics_curated' if firings_to_save == 'firings_curated' else 'cluster_metrics'
kwkf.mda_to_kwik(exp_struct['files']['kwd'],
                 exp_struct['files']['kwik'],
                 file_paths[firings_to_save],
                file_paths[metrics_to_save])


2019-02-22 17:45:07,691 - pipefinch.h5tools.kwik.kwikfunctions - INFO - Creating kwik file /media/zinch/Windows/experiment/p14r14/ephys/kwik/2019-02-15_3125_0102/spikes.kwik from firings /media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-15_3125_0102/sort_out/firings_curated.mda


In [19]:
### extract all unit waveforms
from pipefinch.neural import units
units.get_all_unit_waveforms(exp_struct['files']['kwik'], exp_struct['files']['kwd'])

2019-02-22 17:45:12,635 - pipefinch.neural.units - INFO - About to get all waveforms for 54 units in file /media/zinch/Windows/experiment/p14r14/ephys/kwik/2019-02-15_3125_0102/spikes.kwik


0

In [ ]:
def msort_cleanup(exp_struct: dict):
    # remove the mda files and try to cleanup the msort temp location
    mda_raw_path = exp_struct['files']['mda_raw']
    logger.info('removing intermediate msort mda file {}'.format(mda_raw_path))
    os.remove(mda_raw_path)

def msort_tmp_clean():
    tmp_dir = os.path.abspath(os.environ['ML_TEMPORARY_DIRECTORY'])
    logger.info('Cleaning up msort temp dir {}'.format(tmp_dir))
    
#msort_tmp_clean()
msort_cleanup(exp_struct)

In [165]:
 exp_struct['files']

{'par': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-13_1750_01/params.json',
 'set': '/mnt/zuperfinchjr/Data/p14r14/ephys/raw/2019-02-13_1750_01/settings.isf',
 'kwd': '/media/zinch/Windows/experiment/p14r14/ephys/kwik/2019-02-13_1750_01/streams.kwd',
 'kwik': '/media/zinch/Windows/experiment/p14r14/ephys/kwik/2019-02-13_1750_01/spikes.kwik',
 'kwe': '/media/zinch/Windows/experiment/p14r14/ephys/kwik/2019-02-13_1750_01/events.kwe',
 'mda_raw': '/media/zinch/Windows/experiment/p14r14/ephys/msort/2019-02-13_1750_01/raw.mda'}